In [1]:
import os

list_file = sorted(list(os.walk('./file/3.part'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

list_file

['crawler_1.xlsx',
 'crawler_10.xlsx',
 'crawler_11.xlsx',
 'crawler_12.xlsx',
 'crawler_13.xlsx',
 'crawler_14.xlsx',
 'crawler_15.xlsx',
 'crawler_16.xlsx',
 'crawler_17.xlsx',
 'crawler_18.xlsx',
 'crawler_19.xlsx',
 'crawler_2.xlsx',
 'crawler_20.xlsx',
 'crawler_21.xlsx',
 'crawler_22.xlsx',
 'crawler_23.xlsx',
 'crawler_24.xlsx',
 'crawler_25.xlsx',
 'crawler_26.xlsx',
 'crawler_27.xlsx',
 'crawler_28.xlsx',
 'crawler_29.xlsx',
 'crawler_3.xlsx',
 'crawler_30.xlsx',
 'crawler_31.xlsx',
 'crawler_32.xlsx',
 'crawler_33.xlsx',
 'crawler_34.xlsx',
 'crawler_35.xlsx',
 'crawler_36.xlsx',
 'crawler_37.xlsx',
 'crawler_38.xlsx',
 'crawler_39.xlsx',
 'crawler_4.xlsx',
 'crawler_40.xlsx',
 'crawler_41.xlsx',
 'crawler_42.xlsx',
 'crawler_43.xlsx',
 'crawler_5.xlsx',
 'crawler_6.xlsx',
 'crawler_7.xlsx',
 'crawler_8.xlsx',
 'crawler_9.xlsx']

In [2]:
file = list_file[0]

file

'crawler_1.xlsx'

In [3]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

In [4]:
input_ = pd.read_excel(f'./file/3.part/{file}',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)
input_['Vehicle Page'] = input_['Vehicle Page'].astype(int)

print(f'总数量：{len(input_)}')
input_['Part Number'].tolist()

总数量：1033


['CU10',
 'CU1037',
 'CU1051',
 'CU1052',
 'CU1062',
 'CU1079',
 'CU109',
 'CU1094',
 'CU1108',
 'CU1115',
 'CU1124',
 'CU1125',
 'CU1130',
 'CU1135',
 'CU1140',
 'CU1145',
 'CU1155',
 'CU1164',
 'CU1165',
 'CU1166',
 'CU1174',
 'CU1182',
 'CU1183',
 'CU1190',
 'CU1193',
 'CU1195',
 'CU1198',
 'CU1202',
 'CU1210',
 'CU1212',
 'CU1239',
 'CU1240',
 'CU1242',
 'CU1244',
 'CU1273',
 'CU1278',
 'CU1279',
 'CU1286',
 'CU1290',
 'CU1295',
 'CU1298',
 'CU130',
 'CU13000',
 'CU13001',
 'CU13002',
 'CU13004',
 'CU13005',
 'CU13007',
 'CU13015',
 'CU13016',
 'CU13017',
 'CU13018',
 'CU13019',
 'CU1302',
 'CU13022',
 'CU13022A',
 'CU13024',
 'CU13025',
 'CU13026',
 'CU1303',
 'CU13030',
 'CU13031',
 'CU13035',
 'CU13037',
 'CU13039',
 'CU13041',
 'CU13042',
 'CU13043',
 'CU13044',
 'CU13045',
 'CU13047',
 'CU13049',
 'CU13057',
 'CU13058',
 'CU13060',
 'CU13063',
 'CU13065',
 'CU13067',
 'CU13068',
 'CU1307',
 'CU13070',
 'CU13071',
 'CU13074',
 'CU13076',
 'CU13077',
 'CU13078',
 'CU13079',
 'CU

In [5]:
a = 0

a

0

In [6]:
crawler_status = 'error'

crawler_status

'error'

In [7]:
import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

df_temp, h3_last, page = pd.DataFrame(), '', 1
while page <= input_.loc[a, 'Vehicle Page']:
    b = 0
    while True:
        b += 1
        print(b, page)
        try:
            resp = requests.get(f'''{input_.loc[a, 'Url']}?page={page}#applications''',
                                headers=get_header(),
                                proxies=get_proxy(),
                                timeout=(10, 10))

            if resp.status_code == 200:
                soup = BeautifulSoup(resp.text, 'lxml')
                html = etree.HTML(str(soup))
                
                if html.xpath('//h1[@class="part__titre-code"]/text()')[0].strip() == input_.loc[a, 'Part Number']:
                    break
        except KeyboardInterrupt:
            break
        except:
            continue

    # = = = = = = = = = = = = = = =

    list_h3 = [h3.strip() for h3 in html.xpath('//section[@id="applications"]/h3[@class="part__titre-groupe-vehicules"]/text()')]

    # = = = = = = = = = = = = = = =

    list_table = html.xpath('//section[@id="applications"]/table[@class="part__table-applications"]')

    # = = = = = = = = = = = = = = =

    len(list_table) == 0

    # = = = = = = = = = = = = = = =

    df_temp_temp = pd.DataFrame()
    if len(list_h3) == len(list_table):
        for i, h3, table in zip(range(len(list_h3)), list_h3, list_table):
            list_th = [th.xpath('./text()')[0].strip() if th.xpath('./text()') else '' for th in table.xpath('./thead/tr/th')]
            while '' in list_th:
                list_th.remove('')

            list_tr = table.xpath('./tbody/tr')
            for j, tr in zip(range(len(list_tr)), list_tr):
                list_td = [td.strip() for td in tr.xpath('./td[@class="part__table-applications__data"]/text()')]

                df_temp_temp = pd.concat([df_temp_temp, pd.DataFrame([[input_.loc[a, 'No'], input_.loc[a, 'Part Number'], input_.loc[a, 'Url'], page, i+1, h3, j+1] + list_td], columns=['No', 'Part Number', 'Url', 'Page', 'Make Model Engine No', 'Make Model Engine', 'Row'] + list_th)], ignore_index=True).fillna('')
    elif len(list_h3) + 1 == len(list_table):
        list_tr = list_table[0].xpath('./tbody/tr')
        for j, tr in zip(range(len(list_tr)), list_tr):
            list_td = [td.strip() for td in tr.xpath('./td[@class="part__table-applications__data"]/text()')]

            df_temp_temp = pd.concat([df_temp_temp, pd.DataFrame([[input_.loc[a, 'No'], input_.loc[a, 'Part Number'], input_.loc[a, 'Part Number'], page, 1, h3_last, j+1] + list_td], columns=['No', 'Part Number', 'Url', 'Page', 'Make Model Engine No', 'Make Model Engine', 'Row'] + list_th)], ignore_index=True).fillna('')

        if len(list_table) > 1:
            for i, h3, table in zip(range(len(list_h3)), list_h3, list_table[1:]):
                list_th = [th.xpath('./text()')[0].strip() if th.xpath('./text()') else '' for th in table.xpath('./thead/tr/th')]
                while '' in list_th:
                    list_th.remove('')

                list_tr = table.xpath('./tbody/tr')
                for j, tr in zip(range(len(list_tr)), list_tr):
                    list_td = [td.strip() for td in tr.xpath('./td[@class="part__table-applications__data"]/text()')]

                    df_temp_temp = pd.concat([df_temp_temp, pd.DataFrame([[input_.loc[a, 'No'], input_.loc[a, 'Part Number'], input_.loc[a, 'Part Number'], page, i+2, h3, j+1] + list_td], columns=['No', 'Part Number', 'Url', 'Page', 'Make Model Engine No', 'Make Model Engine', 'Row'] + list_th)], ignore_index=True).fillna('')
    else:
        print(len(list_h3), len(list_table))

    df_temp = pd.concat([df_temp, df_temp_temp], ignore_index=True).fillna('')

    # = = = = = = = = = = = = = = =

    h3_last = list_h3[-1]

    page += 1

    # = = = = = = = = = = = = = = =

df_temp

1 1
1 2


,No,Part Number,Url,Page,Make Model Engine No,Make Model Engine,Row,Year,Fuel,Fuel Del.,ASP.,Eng. VIN,Eng. Desg.,CC,CID
0,1,CU10,https://ecat.spectrapremium.com/en/parts/CU10,1,1,Honda Civic Base L4 1.5L,1,1991,GAS,FI,N,-,D15B2,1493,-
1,1,CU10,https://ecat.spectrapremium.com/en/parts/CU10,1,1,Honda Civic Base L4 1.5L,2,1990,GAS,FI,N,-,D15B2,1493,-
2,1,CU10,https://ecat.spectrapremium.com/en/parts/CU10,1,1,Honda Civic Base L4 1.5L,3,1989,GAS,FI,N,-,D15B2,1493,-
3,1,CU10,https://ecat.spectrapremium.com/en/parts/CU10,1,1,Honda Civic Base L4 1.5L,4,1988,GAS,FI,N,-,D15B2,1493,-
4,1,CU10,https://ecat.spectrapremium.com/en/parts/CU10,1,2,Honda Civic CX L4 1.5L,1,1991,GAS,FI,N,-,D15B2,1493,-
5,1,CU10,https://ecat.spectrapremium.com/en/parts/CU10,1,2,Honda Civic CX L4 1.5L,2,1990,GAS,FI,N,-,D15B2,1493,-
6,1,CU10,https://ecat.spectrapremium.com/en/parts/CU10,1,2,Honda Civic CX L4 1.5L,3,1989,GAS,FI,N,-,D15B2,1493,-
7,1,CU10,https://ecat.spectrapremium.com/en/parts/CU10,1,3,Honda Civic DX L4 1.5L,1,1991,GAS,FI,N,-,D15B2,1493,-
8,1,CU10,https://ecat.spectrapremium.com/en/parts/CU10,1,3,Honda Civic DX L4 1.5L,2,1990,GAS,FI,N,-,D15B2,1493,-
9,1,CU10,https://ecat.spectrapremium.com/en/parts/CU10,1,3,Honda Civic DX L4 1.5L,3,1989,GAS,FI,N,-,D15B2,1493,-


In [8]:
len(df_temp) == 0

False

In [9]:
output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

output_correct

,No,Part Number,Url,Page,Make Model Engine No,Make Model Engine,Row,Year,Fuel,Fuel Del.,ASP.,Eng. VIN,Eng. Desg.,CC,CID
0,1,CU10,https://ecat.spectrapremium.com/en/parts/CU10,1,1,Honda Civic Base L4 1.5L,1,1991,GAS,FI,N,-,D15B2,1493,-
1,1,CU10,https://ecat.spectrapremium.com/en/parts/CU10,1,1,Honda Civic Base L4 1.5L,2,1990,GAS,FI,N,-,D15B2,1493,-
2,1,CU10,https://ecat.spectrapremium.com/en/parts/CU10,1,1,Honda Civic Base L4 1.5L,3,1989,GAS,FI,N,-,D15B2,1493,-
3,1,CU10,https://ecat.spectrapremium.com/en/parts/CU10,1,1,Honda Civic Base L4 1.5L,4,1988,GAS,FI,N,-,D15B2,1493,-
4,1,CU10,https://ecat.spectrapremium.com/en/parts/CU10,1,2,Honda Civic CX L4 1.5L,1,1991,GAS,FI,N,-,D15B2,1493,-
5,1,CU10,https://ecat.spectrapremium.com/en/parts/CU10,1,2,Honda Civic CX L4 1.5L,2,1990,GAS,FI,N,-,D15B2,1493,-
6,1,CU10,https://ecat.spectrapremium.com/en/parts/CU10,1,2,Honda Civic CX L4 1.5L,3,1989,GAS,FI,N,-,D15B2,1493,-
7,1,CU10,https://ecat.spectrapremium.com/en/parts/CU10,1,3,Honda Civic DX L4 1.5L,1,1991,GAS,FI,N,-,D15B2,1493,-
8,1,CU10,https://ecat.spectrapremium.com/en/parts/CU10,1,3,Honda Civic DX L4 1.5L,2,1990,GAS,FI,N,-,D15B2,1493,-
9,1,CU10,https://ecat.spectrapremium.com/en/parts/CU10,1,3,Honda Civic DX L4 1.5L,3,1989,GAS,FI,N,-,D15B2,1493,-


In [10]:
crawler_status = 'ok'

crawler_status

'ok'

In [11]:
if crawler_status == 'error':
    output_error = pd.concat([output_error, pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)], ignore_index=True).fillna('')

output_error

""


In [12]:
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['No', 'Page', 'Make Model Engine No', 'Row'],
                                                ascending=[True, True, True, True],
                                                ignore_index=True)
    output_correct.to_excel('./test_vehicle.xlsx', index=False)

output_correct

,No,Part Number,Url,Page,Make Model Engine No,Make Model Engine,Row,Year,Fuel,Fuel Del.,ASP.,Eng. VIN,Eng. Desg.,CC,CID
0,1,CU10,https://ecat.spectrapremium.com/en/parts/CU10,1,1,Honda Civic Base L4 1.5L,1,1991,GAS,FI,N,-,D15B2,1493,-
1,1,CU10,https://ecat.spectrapremium.com/en/parts/CU10,1,1,Honda Civic Base L4 1.5L,2,1990,GAS,FI,N,-,D15B2,1493,-
2,1,CU10,https://ecat.spectrapremium.com/en/parts/CU10,1,1,Honda Civic Base L4 1.5L,3,1989,GAS,FI,N,-,D15B2,1493,-
3,1,CU10,https://ecat.spectrapremium.com/en/parts/CU10,1,1,Honda Civic Base L4 1.5L,4,1988,GAS,FI,N,-,D15B2,1493,-
4,1,CU10,https://ecat.spectrapremium.com/en/parts/CU10,1,2,Honda Civic CX L4 1.5L,1,1991,GAS,FI,N,-,D15B2,1493,-
5,1,CU10,https://ecat.spectrapremium.com/en/parts/CU10,1,2,Honda Civic CX L4 1.5L,2,1990,GAS,FI,N,-,D15B2,1493,-
6,1,CU10,https://ecat.spectrapremium.com/en/parts/CU10,1,2,Honda Civic CX L4 1.5L,3,1989,GAS,FI,N,-,D15B2,1493,-
7,1,CU10,https://ecat.spectrapremium.com/en/parts/CU10,1,3,Honda Civic DX L4 1.5L,1,1991,GAS,FI,N,-,D15B2,1493,-
8,1,CU10,https://ecat.spectrapremium.com/en/parts/CU10,1,3,Honda Civic DX L4 1.5L,2,1990,GAS,FI,N,-,D15B2,1493,-
9,1,CU10,https://ecat.spectrapremium.com/en/parts/CU10,1,3,Honda Civic DX L4 1.5L,3,1989,GAS,FI,N,-,D15B2,1493,-


In [13]:
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel('./test_vehicle_error.xlsx', index=False)

output_error

""
